In [1]:
import re
import os
import nltk
import spacy
import string
import psutil
import subprocess
import fileinput
import contractions
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import dataset_utils
import sentiment_utils
import absa_utils

from itertools import chain
from tqdm import tqdm
from afinn import Afinn
from wordcloud import WordCloud
from collections import Counter
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.punkt import PunktLanguageVars
from nltk.stem import WordNetLemmatizer
from scipy.sparse import vstack
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfTransformer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.corpus import sentiwordnet as swn
from subprocess import Popen, PIPE, STDOUT

#nltk.download('sentiwordnet')
#nltk.download('punkt')

No CUDA GPU found in your device
[2024-02-08 17:26:37] (2.4.0) PyABSA(2.4.0): If your code crashes on Colab, please use the GPU runtime. Then run "pip install pyabsa[dev] -U" and restart the kernel.
Or if it does not work, you can use v1.x versions, e.g., pip install pyabsa<2.0 -U




Try to downgrade transformers<=4.29.0.






# Constants

In [2]:
PREPROCESSED_DATASET_PATH = "./dataset/dataset.csv"
ABSA_REVIEWS_PATH = "./dataset/absa_reviews.csv"
ABSA_DATASET_PATH = "./dataset/absa_dataset.csv"
DATASET_WITH_SENTIMENT_PATH = "./dataset/dataset_with_sentiment.csv"
ASUM_BIN = "./asum/bin"
ASUM_INPUT = "./asum/input"
ASUM_OUTPUT =  "./asum/output"

# Import dataset

In [3]:
dataset = pd.read_csv(DATASET_WITH_SENTIMENT_PATH)
dataset

c:\Users\raffa\anaconda3\envs\SCMS\Lib\multiprocessing\pool.py:268: ResourceWarning: unclosed running multiprocessing pool <multiprocessing.pool.Pool state=RUN pool_size=1>
  _warn(f"unclosed running multiprocessing pool {self!r}",
C:\Users\raffa\AppData\Local\Temp\ipykernel_24628\3537435434.py:1: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(DATASET_WITH_SENTIMENT_PATH)


,rating,opinion,predictedSentiment,numericSentiment,reviewTime,unixReviewTime,reviewText,reviewLength,summary,summaryLength,...,title,description,price,reviewerID,reviewerName,verified,vote,style,image,imageURLHighRes
0,5,positive,positive,1,2010-02-14,1266105600,I usually keep a few of these around for a qui...,78,Preferred Brand,2,...,"Dole Mandarin in Orange Gel, 4.3-Ounce Cups (...",['Mandarin Orange Gel individual serving in pl...,34.08,A3BH49ZKESHDID,Fred Camfield,False,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...
1,5,positive,positive,1,2015-12-16,1450224000,"Fits nice in lunch pack, clean water no taste....",10,clean water no taste. H2O,5,...,"Icelandic Glacial Natural Spring Water, 500 Mi...",['Icelandic glacial natural spring water from ...,35.99,A3PJYWWN9DHVIH,John,True,NaN,"{'Size:': ' 330 Milliliter', 'Flavor:': ' 30 C...",NaN,['https://images-na.ssl-images-amazon.com/imag...
2,5,positive,positive,1,2016-03-25,1458864000,We are vegan. We use this to thicken up sauces...,12,Five Stars,2,...,Guar Gum 8 Ounces (Pack of 2),"[""Pure PowderNatural ThickenerA Gluten-Free Fo...",14.97,A1NCD09GCSJGZD,Test Products &amp;amp;amp; Avid Reader,True,NaN,NaN,NaN,[]
3,2,negative,negative,-1,2017-08-08,1502150400,I ordered these mints 2 times before & loved t...,60,"Need to exchange, if possible",5,...,Sugar-Free Mints - Peppermint Flavor (100 tabs...,"['', '<strong>Features</strong>', '', '', '', ...",5.75,A3AZ2PWY6UULE8,Patricia L. Titzer,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...
4,4,positive,positive,1,2013-06-01,1370044800,"Price is ok, taste is good. Well worth the ti...",44,Got it for long term storage.,6,...,"Keystone Meats All Natural Canned Pork, 28 Ounce",['Keystone Meats is an OHIO PROUD United State...,11.97,A3MDA891A1EPT8,Chris W.,False,5.0,{'Size:': ' 14.5 Ounce'},NaN,['https://images-na.ssl-images-amazon.com/imag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752149,5,positive,positive,1,2013-08-30,1377820800,We have tried all of the nut mixes. This one i...,31,Probably the best tasting mixes,5,...,"Planters Deluxe Mixed Nuts, Salted, 34 Ounce Jar","[""Nuts are the heroes of wholesome snacking. A...",17.98,A31UVXEANHGDSL,DavidA,True,NaN,NaN,NaN,[]
752150,5,positive,positive,1,2015-07-27,1437955200,mmmm,1,Five Stars,2,...,REESE'S Pieces Peanut Butter Candy (Pack of 18),"[""Make snack time sweet with REESE'S PIECES Ca...",35.09,AI5YDJXUZUTJK,CB63SK,True,NaN,{'Size:': ' 10.5 Ounce (Pack of 6)'},NaN,['https://images-na.ssl-images-amazon.com/imag...
752151,5,positive,positive,1,2016-04-26,1461628800,these candies are amazing! The lemon is delici...,32,delicious candies!!,2,...,Hand Crafted Candy Tin Lemon Ice Chips Candy 1...,"['Ice chips Candy Lemon, 1. 76 oz. Made out of...",5.77,A1WOAM7BQ0YR43,Stephanie,False,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...
752152,4,positive,positive,1,2014-11-23,1416700800,Delicious.,1,Four Stars,2,...,"Kettle Brand Potato Chips, Jalapeno, Single-Se...",['Jalapeno Kettle Brand Potato Chips have a ze...,29.98,A2YSFUH31GVNCX,Norman E. Paddock,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...


# Create ASUM dataset

In [4]:
#dataset = dataset[dataset['opinion'] != 'neutral']
#dataset

In [5]:
new_dataset = absa_utils.preprocess_asum_dataset(dataset)
new_dataset

100%|██████████| 752154/752154 [00:10<00:00, 68438.81it/s] 


,rating,opinion,predictedSentiment,numericSentiment,reviewTime,unixReviewTime,reviewText,reviewLength,summary,summaryLength,...,reviewerID,reviewerName,verified,vote,style,image,imageURLHighRes,sentences,prep_sentences,prep_sentences_ids
0,5,positive,positive,1,2010-02-14,1266105600,I usually keep a few of these around for a qui...,78,Preferred Brand,2,...,A3BH49ZKESHDID,Fred Camfield,False,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,[I usually keep a few of these around for a qu...,"[[usual, keep, around, quick, snack], [packag,...","[[54981, 27343, 2660, 41368, 47120], [37103, 2..."
1,5,positive,positive,1,2015-12-16,1450224000,"Fits nice in lunch pack, clean water no taste....",10,clean water no taste. H2O,5,...,A3PJYWWN9DHVIH,John,True,NaN,"{'Size:': ' 330 Milliliter', 'Flavor:': ' 30 C...",NaN,['https://images-na.ssl-images-amazon.com/imag...,"[Fits nice in lunch pack, clean water no taste...","[[fit, nice, lunch, pack, clean, water, tast],...","[[18496, 34532, 30179, 37097, 9823, 56367, 507..."
2,5,positive,positive,1,2016-03-25,1458864000,We are vegan. We use this to thicken up sauces...,12,Five Stars,2,...,A1NCD09GCSJGZD,Test Products &amp;amp;amp; Avid Reader,True,NaN,NaN,NaN,[],"[We are vegan., We use this to thicken up sauc...","[[vegan], [use, thicken, sauc, make]]","[[55321], [54920, 51793, 44649, 30598]]"
3,2,negative,negative,-1,2017-08-08,1502150400,I ordered these mints 2 times before & loved t...,60,"Need to exchange, if possible",5,...,A3AZ2PWY6UULE8,Patricia L. Titzer,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,[I ordered these mints 2 times before & loved ...,"[[order, mint, time, befor, love, time, made, ...","[[36363, 32446, 52215, 4376, 29970, 52215, 304..."
4,4,positive,positive,1,2013-06-01,1370044800,"Price is ok, taste is good. Well worth the ti...",44,Got it for long term storage.,6,...,A3MDA891A1EPT8,Chris W.,False,5.0,{'Size:': ' 14.5 Ounce'},NaN,['https://images-na.ssl-images-amazon.com/imag...,"[Price is ok, taste is good., Well worth the t...","[[price, ok, tast, good], [well, worth, time, ...","[[40256, 35885, 50798, 21388], [56611, 57584, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
752149,5,positive,positive,1,2013-08-30,1377820800,We have tried all of the nut mixes. This one i...,31,Probably the best tasting mixes,5,...,A31UVXEANHGDSL,DavidA,True,NaN,NaN,NaN,[],"[We have tried all of the nut mixes., This one...","[[tri, nut, mix], [one, probabl, best, tast], ...","[[53086, 35270, 32642], [36082, 40394, 4682, 5..."
752150,5,positive,positive,1,2015-07-27,1437955200,mmmm,1,Five Stars,2,...,AI5YDJXUZUTJK,CB63SK,True,NaN,{'Size:': ' 10.5 Ounce (Pack of 6)'},NaN,['https://images-na.ssl-images-amazon.com/imag...,[mmmm],[[mm]],[[32711]]
752151,5,positive,positive,1,2016-04-26,1461628800,these candies are amazing! The lemon is delici...,32,delicious candies!!,2,...,A1WOAM7BQ0YR43,Stephanie,False,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,"[these candies are amazing!, The lemon is deli...","[[candi, amaz], [lemon, delici, remind, lemon,...","[[7488, 1536], [28993, 13269, 42649, 28993, 14..."
752152,4,positive,positive,1,2014-11-23,1416700800,Delicious.,1,Four Stars,2,...,A2YSFUH31GVNCX,Norman E. Paddock,True,NaN,NaN,NaN,['https://images-na.ssl-images-amazon.com/imag...,[Delicious.],[[delici]],[[13269]]


# Execute ASUM

In [6]:
num_sentiments = str(2)
num_topics = str(10)
num_iterations = str(1000)
num_threads = str(psutil.cpu_count())

alpha = "0.1" 
beta = "0.001/0.1/0"
gamma = "1/1"

In [7]:
result = subprocess.run("cd", capture_output=True, shell=True)
current_directory = result.stdout.decode('cp1252').strip()
asum_bin = os.path.join(current_directory, r"asum\bin")
print(asum_bin)

c:\Users\raffa\Downloads\data_analytics_project\asum\bin


In [8]:
asum_input = os.path.join(current_directory, r"asum\input")
asum_output = os.path.join(current_directory, r"asum\output")
print(asum_input)
print(asum_output)

c:\Users\raffa\Downloads\data_analytics_project\asum\input
c:\Users\raffa\Downloads\data_analytics_project\asum\output


In [9]:
command = ' '.join([os.path.join(asum_bin, 'asum.sh'), num_sentiments, num_topics, num_iterations, num_threads, alpha, beta, gamma, asum_input, asum_output])
command

'c:\\Users\\raffa\\Downloads\\data_analytics_project\\asum\\bin\\asum.sh 2 10 1000 16 0.1 0.001/0.1/0 1/1 c:\\Users\\raffa\\Downloads\\data_analytics_project\\asum\\input c:\\Users\\raffa\\Downloads\\data_analytics_project\\asum\\output'

In [10]:
p = Popen(command, stdin=PIPE, stdout=PIPE, stderr=STDOUT, shell=True, cwd=ASUM_BIN)
for line in p.stdout:
    print(line)
    print('\r', line.decode(), end='')

# Result analysis

In [23]:
output_path = "./asum/output/STO2-T10-S2(2)-A0.1-B0.001,0.1,0.0-G1.0,1.0-I1000-ProbWords.csv"

In [24]:
with fileinput.FileInput(output_path, inplace=True, backup='.bak') as file:
    for riga in file:
        riga_modificata = riga.replace(',', '.')
        print(riga_modificata, end='')

with fileinput.FileInput(output_path, inplace=True, backup='.bak') as file:
    for riga in file:
        riga_modificata = riga.replace(').', '),')
        print(riga_modificata, end='')
        
with open(output_path, 'r') as file:
    lines = file.readlines()

lines[0] = lines[0].replace('.', ',')

with open(output_path, 'w') as file:
    file.writelines(lines)

In [25]:
df = pd.read_csv(output_path, index_col=False)
df

,S0-T0,S0-T1,S0-T2,S0-T3,S0-T4,S0-T5,S0-T6,S0-T7,S0-T8,S0-T9,...,S1-T1,S1-T2,S1-T3,S1-T4,S1-T5,S1-T6,S1-T7,S1-T8,S1-T9,Unnamed: 20
0,tast (0.024),use (0.026),coffe (0.080),flavor (0.019),great (0.030),good (0.030),tea (0.092),love (0.035),good (0.019),great (0.105),...,day (0.013),water (0.023),chocol (0.029),tast (0.030),tast (0.059),store (0.034),sugar (0.021),order (0.039),bag (0.025),NaN
1,flavor (0.023),oil (0.025),flavor (0.026),use (0.018),snack (0.029),price (0.029),flavor (0.023),good (0.015),use (0.016),good (0.098),...,drink (0.013),use (0.022),like (0.029),like (0.028),like (0.040),buy (0.029),ingredi (0.013),time (0.029),packag (0.016),NaN
2,good (0.021),salt (0.019),good (0.020),good (0.015),good (0.029),great (0.022),tast (0.019),candi (0.013),tast (0.016),product (0.062),...,use (0.013),cup (0.014),tast (0.024),sugar (0.025),flavor (0.037),price (0.027),fat (0.013),buy (0.021),box (0.014),NaN
3,love (0.021),good (0.018),tast (0.020),like (0.014),bar (0.024),product (0.020),love (0.018),great (0.012),great (0.016),veri (0.054),...,help (0.011),make (0.013),delici (0.022),flavor (0.024),veri (0.020),local (0.019),calori (0.012),one (0.017),open (0.013),NaN
4,great (0.020),tast (0.017),like (0.018),tast (0.014),love (0.022),buy (0.018),good (0.017),like (0.012),make (0.015),tast (0.047),...,eat (0.011),add (0.011),flavor (0.018),use (0.020),littl (0.011),amazon (0.018),contain (0.010),bag (0.013),one (0.012),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,thing (0.002),regular (0.002),think (0.002),powder (0.002),give (0.002),think (0.002),fresh (0.002),know (0.002),first (0.002),mm (0.001),...,healthi (0.002),rins (0.002),amount (0.002),free (0.002),feel (0.002),health (0.002),better (0.002),say (0.002),easili (0.002),NaN
96,brand (0.002),still (0.002),bought (0.002),jerki (0.002),bit (0.002),mani (0.002),everi (0.002),ginger (0.002),stuff (0.002),shape (0.001),...,benefit (0.002),packag (0.002),enough (0.002),bad (0.002),last (0.002),delici (0.002),even (0.002),quick (0.002),onc (0.002),NaN
97,lot (0.002),extra (0.002),lot (0.002),ever (0.002),doe (0.002),sinc (0.002),honey (0.002),gave (0.002),never (0.002),honey (0.001),...,sever (0.002),delici (0.002),go (0.002),appl (0.002),enough (0.002),stuff (0.002),peopl (0.002),think (0.002),serv (0.002),NaN
98,real (0.002),ad (0.002),give (0.002),enjoy (0.002),look (0.002),wish (0.002),morn (0.002),size (0.002),crust (0.002),much (0.001),...,differ (0.002),cool (0.002),coconut (0.002),never (0.002),chemic (0.002),sold (0.002),chocol (0.002),still (0.002),dri (0.002),NaN


In [26]:
positive_topics = df.iloc[:, :int(num_topics)].head(10)
positive_topics

,S0-T0,S0-T1,S0-T2,S0-T3,S0-T4,S0-T5,S0-T6,S0-T7,S0-T8,S0-T9
0,tast (0.024),use (0.026),coffe (0.080),flavor (0.019),great (0.030),good (0.030),tea (0.092),love (0.035),good (0.019),great (0.105)
1,flavor (0.023),oil (0.025),flavor (0.026),use (0.018),snack (0.029),price (0.029),flavor (0.023),good (0.015),use (0.016),good (0.098)
2,good (0.021),salt (0.019),good (0.020),good (0.015),good (0.029),great (0.022),tast (0.019),candi (0.013),tast (0.016),product (0.062)
3,love (0.021),good (0.018),tast (0.020),like (0.014),bar (0.024),product (0.020),love (0.018),great (0.012),great (0.016),veri (0.054)
4,great (0.020),tast (0.017),like (0.018),tast (0.014),love (0.022),buy (0.018),good (0.017),like (0.012),make (0.015),tast (0.047)
5,use (0.020),great (0.016),cup (0.015),great (0.013),tast (0.020),amazon (0.016),like (0.014),flavor (0.012),love (0.015),price (0.038)
6,chocol (0.016),coconut (0.014),love (0.014),sauc (0.013),eat (0.013),love (0.016),great (0.012),one (0.010),like (0.014),qualiti (0.022)
7,like (0.015),love (0.013),roast (0.013),love (0.011),like (0.011),order (0.012),drink (0.012),tast (0.009),free (0.014),love (0.022)
8,milk (0.014),like (0.010),great (0.013),make (0.011),veri (0.011),veri (0.011),veri (0.011),gift (0.008),gluten (0.013),excel (0.021)
9,make (0.013),popcorn (0.010),one (0.011),soup (0.009),one (0.010),best (0.010),green (0.011),chocol (0.007),bread (0.013),fresh (0.020)


In [27]:
negative_topics = df.iloc[:, int(num_topics):2*int(num_topics)].head(10)
negative_topics

,S1-T0,S1-T1,S1-T2,S1-T3,S1-T4,S1-T5,S1-T6,S1-T7,S1-T8,S1-T9
0,product (0.024),day (0.013),water (0.023),chocol (0.029),tast (0.030),tast (0.059),store (0.034),sugar (0.021),order (0.039),bag (0.025)
1,review (0.021),drink (0.013),use (0.022),like (0.029),like (0.028),like (0.040),buy (0.029),ingredi (0.013),time (0.029),packag (0.016)
2,star (0.010),use (0.013),cup (0.014),tast (0.024),sugar (0.025),flavor (0.037),price (0.027),fat (0.013),buy (0.021),box (0.014)
3,would (0.010),help (0.011),make (0.013),delici (0.022),flavor (0.024),veri (0.020),local (0.019),calori (0.012),one (0.017),open (0.013)
4,tri (0.008),eat (0.011),add (0.011),flavor (0.018),use (0.020),littl (0.011),amazon (0.018),contain (0.010),bag (0.013),one (0.012)
5,one (0.008),get (0.010),cook (0.010),peanut (0.017),sweet (0.014),sweet (0.010),groceri (0.014),protein (0.010),tri (0.013),like (0.010)
6,like (0.008),like (0.009),minut (0.010),veri (0.015),water (0.013),smell (0.009),get (0.014),product (0.009),box (0.013),use (0.008)
7,amazon (0.007),one (0.008),mix (0.009),butter (0.013),sweeten (0.012),realli (0.009),find (0.013),serv (0.008),would (0.012),contain (0.008)
8,compani (0.007),work (0.007),put (0.009),sweet (0.013),drink (0.011),bit (0.009),much (0.013),high (0.008),purchas (0.011),veri (0.008)
9,becaus (0.006),time (0.007),time (0.008),nut (0.012),much (0.010),doe (0.009),better (0.012),sodium (0.008),last (0.011),plastic (0.008)


In [22]:
first_positive_topic = positive_topics['S0-T0'].values
first_positive_topic

array(['chocol (0.034)', 'tast (0.025)', 'flavor (0.025)', 'good (0.024)',
       'love (0.024)', 'like (0.020)', 'bar (0.019)', 'peanut (0.016)',
       'butter (0.012)', 'great (0.012)'], dtype=object)